In [1]:
import os
os.environ["RUN_MODE"] = "0"

In [2]:
from DataLoader import *
from Data import Data
from Evalute import evaluate_model
from Models.SimpleCF import SimpleCF
import numpy as np

Dataset path - Running mode: 0


In [3]:
from time import time
# An example for running the model and evaluating using leave-1-out and top-k using hit ratio and NCDG metrics
convert_binary = True
load_model = False
testset_percentage = 0.2


print('Started...')

df = get_movielens100k(convert_binary=False)

data = Data(df, seed=42)
t0 = time()
training_set, test_set, n_users, n_movies = data.pre_processing(test_percent=0.5, train_precent=1)

Started...
n_users: 943 n_movies: 1682
get_train_instances done


In [4]:
# WORK ON EMBEDDINGS

n_fake_users = 300
# possible the need to have n_movies+1
fake_users_mat = np.zeros((n_fake_users , n_movies))
n_users_w_fake = n_fake_users + n_users


In [5]:
epochs = 1

low_rank_cf_model = SimpleCF()

best_hr=0
best_ndcg = 0
low_rank_cf_model.set_model(n_users, n_movies, n_latent_factors=64)
t0 = time()
mean_hr, mean_ndcg = evaluate_model(low_rank_cf_model, test_set)
print('Init: HR = %.4f, NDCG = %.4f, Eval:[%.1f s]'
      % (mean_hr, mean_ndcg, time()-t0 ))
for epoch in range(epochs):
    t1 = time()
    loss = low_rank_cf_model.fit_once(training_set, batch_size=128, verbose=0)
    t2 = time()
    mean_hr, mean_ndcg = evaluate_model(low_rank_cf_model, test_set)
    t3 = time()
    print('Iteration: %d Fit:[%.1f s]: HR = %.4f, NDCG = %.4f, loss = %.4f, Eval:[%.1f s]'
          % (epoch+1, t2 - t1, mean_hr, mean_ndcg, loss, t3 - t2))
    if mean_hr > best_hr and mean_ndcg > best_ndcg:
        best_hr = mean_hr
        best_ndcg = mean_ndcg
        low_rank_cf_model.save_model()
print('Total time: [%.1f s]' % (time()- t0))

model has been set
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
movie_input (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 64)        60416       user_input[0][0]                 
__________________________________________________________________________________________________
movie_embedding (Embedding)     (None, 1, 64)        107712      movie_input[0][0]                
___________________________________________________________________________

In [6]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [7]:
tf.executing_eagerly()

False

In [8]:
n_fake_users = 50

In [9]:
"""FGSM ATTACK"""

user_inp = tf.placeholder(tf.float32, (n_fake_users, 64))
item_inp = tf.placeholder(tf.float32, (n_fake_users, 64))
# true_out = tf.placeholder(tf.float32, (n_fake_users, 10))
epsilon = tf.placeholder(tf.float32, [n_fake_users, 64])
# user_inp = tf.Variable((n_fake_users, 64), tf.float32)
# item_inp = tf.Variable((n_fake_users, 64), tf.float32)
# # true_out = tf.placeholder(tf.float32, (n_fake_users, 10))
# epsilon = tf.Variable([n_fake_users, 64], tf.float32)

In [ ]:
train
# test


ratings = low_rank_cf_model.model([user_inp, item_inp])

# [0 0 0 0 0 0 0 0 0 0 0 ] ; [11 545 897 252 45 78 5 4 5 9 6 4] -- > 0, 1 ,0 

In [ ]:
grad = tf.gradients(loss, [user_inp, item_inp])

In [9]:
"""FGSM ATTACK"""

n_adversarial = y_test.shape[0]

trgt_inp = tf.placeholder(tf.float32, (n_fake_users, 64))
# true_out = tf.placeholder(tf.float32, (n_fake_users, 10))
epsilon = tf.placeholder(tf.float32, [n_adversarial, img_width, img_height, 1])

new_inp = tf.identity(trgt_inp)  #returns a tensor with the same shape and type

output = model(new_inp)

loss = tf.sqrt(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=true_out))

grad = tf.gradients(loss, new_inp)
new_inp = tf.stop_gradient(new_inp + epsilon * tf.sign(grad))

step = 0.1
eps = np.full((n_adversarial, img_width, img_height, 1), step)

feed_dict = {trgt_inp: x_test,
             true_out: y_test,
             epsilon: eps}

x_adv, l = sess.run([new_inp, loss],
                    feed_dict)

x_adv = x_adv.reshape((n_adversarial, img_width, img_height, 1))

acc_bfr_attck = model.evaluate(x=x_test, y=y_test, batch_size=32)[1]
acc_aftr_attck = model.evaluate(x=x_adv, y=y_test, batch_size=32)[1]
print('Accuracy before FGSM {}, Accuracy after FGSM {}'.format(acc_bfr_attck, acc_aftr_attck))

NameError: name 'y_test' is not defined

In [7]:
x = [[2.]]
m = tf.matmul(x, x)
print("hello, {}".format(m))


hello, [[4.]]


In [ ]:
tf.Variable